In [ ]:
def s(a):
    a.append(5)
a=[4,5,6]
b=a
s(a)

b

In [2]:
a={4:5}
print(a.get(6))

None


In [4]:
set((4,5,6,6))

{4, 5, 6}

In [ ]:
next(a)

In [ ]:
a=1
k=[a for a in range(5)]
a
import hashlib

In [ ]:
p= 'REPLACE_THIS_WITH_PASSPHRASE'
hashlib.sha224(p.encode('utf-8')).hexdigest()

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go # 导入 plotly

# --- 1. 加载和解析数据 ---
# (这部分和之前的代码完全相同)
file_path = '汇总2.xlsx' 
try:
    df = pd.read_excel(file_path, header=None)
    
    x_labels = df.iloc[1, 2:].values.astype(float)
    y_labels = df.iloc[2:, 1].values.astype(float)
    z_data = df.iloc[2:, 2:].values.astype(float)

    X, Y = np.meshgrid(x_labels, y_labels)
    
    xs = X.flatten()
    ys = Y.flatten()
    zs = z_data.flatten()

    # --- 2. 使用 Plotly 绘制 3D 散点图 ---

    # 创建一个 3D 散点图对象
    fig = go.Figure(data=[go.Scatter3d(
        x=xs,  # X 轴数据
        y=ys,  # Y 轴数据
        z=zs,  # Z 轴数据
        mode='markers', # 模式：只显示点
        marker=dict(
            size=5,                # 点的大小
            color=zs,              # 点的颜色基于 Z 值
            colorscale='Viridis',  # 使用 'Viridis' 色图 (和 matplotlib 一样)
            opacity=0.8,           # 透明度
            colorbar=dict(title='数值') # 添加颜色条
        )
    )])

    # --- 3. 设置图表布局 (标题和坐标轴标签) ---
    fig.update_layout(
        title='FLP 与 Raf 表达量 3D散点图',
        scene=dict(
            xaxis_title='FLP 表达量',
            yaxis_title='Raf 表达量',
            zaxis_title='数值'
        ),
        margin=dict(l=0, r=0, b=0, t=40) # 调整边界
    )

    # --- 4. 显示图表 ---
    # 这将在您的默认浏览器中打开一个可交互的 .html 文件
    # 或者直接在 Jupyter Notebook 的输出单元格中显示
    fig.show()

except FileNotFoundError:
    print(f"错误：未找到文件 '{file_path}'。")
except Exception as e:
    print(f"处理数据时发生错误：{e}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.tree import DecisionTreeRegressor, export_text

# --- 0. 开启 Matplotlib 交互模式 (重要！) ---
# 如果您在 Jupyter/VS Code Notebook, 请在开头运行
# %matplotlib widget 
# (需要先 pip install ipympl)

# --- 1. 配置matplotlib以支持中文 ---
try:
    plt.rcParams['font.sans-serif'] = ['SimHei'] # Windows/Linux
    plt.rcParams['axes.unicode_minus'] = False 
except Exception as e:
    print(f"警告：设置中文字体失败，标签可能无法正常显示。错误: {e}")

# --- 2. 加载和解析数据 ---
file_path = '汇总2.xlsx' 
try:
    df = pd.read_excel(file_path, header=None)
    
    x_labels = df.iloc[1, 2:].values.astype(float)
    y_labels = df.iloc[2:, 1].values.astype(float)
    z_data = df.iloc[2:, 2:].values.astype(float)

    X, Y = np.meshgrid(x_labels, y_labels)
    
    xs = X.flatten() # 展平为 1D 数组
    ys = Y.flatten() # 展平为 1D 数组
    zs = z_data.flatten() # 展平为 1D 数组

    # --- 3. 准备训练数据 ---
    # sklearn 需要的 X (特征) 是一个 (n_samples, n_features) 的2D数组
    # 在这里是 (64, 2)
    X_train = np.stack([xs, ys], axis=1)
    # sklearn 需要的 y (目标) 是一个 (n_samples,) 的1D数组
    y_train = zs

    # --- 4. 训练回归决策树 ---
    # 我们设置一个 max_depth (最大深度) 来防止树长得过深 (过拟合)
    # 您可以调整 max_depth=4 这个值，比如改成 3 或 5 看看效果
    tree_reg = DecisionTreeRegressor(max_depth=7, random_state=42)
    tree_reg.fit(X_train, y_train)

    # --- 5. 【请求一】返回决策树架构 ---
    print("--- 回归决策树架构 (Decision Tree Architecture) ---")
    feature_names = ['FLP 表达量', 'Raf 表达量']
    tree_rules = export_text(tree_reg, feature_names=feature_names)
    print(tree_rules)
    print("--------------------------------------------------")


    # --- 6. 【请求二】将其填充到空间中 (可视化) ---
    
    # a. 创建一个更精细的网格 (grid) 用于绘图
    # 我们将在 x 和 y 的最小/最大值之间各取 100 个点
    x_range = np.linspace(xs.min(), xs.max(), 100)
    y_range = np.linspace(ys.min(), ys.max(), 100)
    
    # b. 创建绘图用的 X, Y 坐标网格
    X_vis, Y_vis = np.meshgrid(x_range, y_range)
    
    # c. 准备用于预测的输入数据 (100*100 = 10000 个点)
    # (10000, 2)
    X_pred_input = np.stack([X_vis.flatten(), Y_vis.flatten()], axis=1)
    
    # d. 使用训练好的模型进行预测
    Z_pred = tree_reg.predict(X_pred_input)
    
    # e. 将预测结果 Z_pred 变回 (100, 100) 的网格形状
    Z_vis = Z_pred.reshape(X_vis.shape)

    # --- 7. 绘制 3D 视图 ---
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')

    # a. 绘制决策树预测的 "曲面" (会是阶梯状)
    #    alpha=0.7 设置透明度，方便看到下面的点
    ax.plot_surface(X_vis, Y_vis, Z_vis, cmap='viridis', alpha=0.7, 
                    rstride=1, cstride=1, edgecolor='none')

    # b. 绘制原始的 64 个数据点 (用红色大点表示)
    ax.scatter(xs, ys, zs, c='red', s=40, depthshade=True, label='原始数据点')

    # --- 8. 设置图表属性 ---
    ax.set_title(f'回归决策树 (max_depth={tree_reg.max_depth}) 拟合曲面', fontsize=18, pad=20)
    ax.set_xlabel('FLP 表达量', fontsize=12, labelpad=15)
    ax.set_ylabel('Raf 表达量', fontsize=12, labelpad=15)
    ax.set_zlabel('预测数值', fontsize=12, labelpad=15)
    ax.legend()

    # 设置一个更合适的视角
    ax.view_init(elev=20, azim=-120) 

    plt.show()

except FileNotFoundError:
    print(f"错误：未找到文件 '{file_path}'。")
except Exception as e:
    print(f"处理数据时发生错误：{e}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# --- 导入 scikit-learn 模块 ---
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

# --- 0. 开启 Matplotlib 交互模式 (重要！) ---
# 如果您在 Jupyter/VS Code Notebook, 请在开头运行
# %matplotlib widget 
# (需要先 pip install ipympl)

# --- 1. 配置matplotlib以支持中文 ---
try:
    plt.rcParams['font.sans-serif'] = ['SimHei'] # Windows/Linux
    plt.rcParams['axes.unicode_minus'] = False 
except Exception as e:
    print(f"警告：设置中文字体失败，标签可能无法正常显示。错误: {e}")

# --- 2. 加载和解析数据 ---
# 注意：使用您提供的特定索引
file_path = '汇总2.xlsx' 

try:
    df = pd.read_excel(file_path, header=None)

    # --- 2b. 稳健的数据提取 ---
    # (这可以防止因表格外的空单元格导致NaN)
    
    # 提取 X 轴标签
    x_labels_raw = df.iloc[1, 2:]
    x_indices = x_labels_raw.notna()
    x_labels = x_labels_raw[x_indices].values.astype(float)
    
    # 提取 Y 轴标签
    y_labels_raw = df.iloc[2:, 1]
    y_indices = y_labels_raw.notna()
    y_labels = y_labels_raw[y_indices].values.astype(float)
    
    # 根据 X 和 Y 的有效索引，提取 Z 数据块
    z_data = df.iloc[y_indices[y_indices].index, 
                     x_indices[x_indices].index].values.astype(float)

    print(f"数据加载成功: X轴 {len(x_labels)} 个点, Y轴 {len(y_labels)} 个点, Z数据 {z_data.shape}")

    # --- 3. 准备通用数据 ---
    
    # a. 创建网格并展平，用于训练
    X_grid, Y_grid = np.meshgrid(x_labels, y_labels)
    xs = X_grid.flatten() # 1D 数组
    ys = Y_grid.flatten() # 1D 数组
    zs = z_data.flatten() # 1D 数组

    # (N, 2) 的特征矩阵
    X_train = np.stack([xs, ys], axis=1)
    # (N,) 的目标向量
    y_train = zs

    # b. 创建精细的可视化网格
    x_range = np.linspace(xs.min(), xs.max(), 100)
    y_range = np.linspace(ys.min(), ys.max(), 100)
    X_vis, Y_vis = np.meshgrid(x_range, y_range)
    # (10000, 2) 的预测输入
    X_pred_input = np.stack([X_vis.flatten(), Y_vis.flatten()], axis=1)

    # --- 4. 数据标准化 (SVR 和 GPR 需要) ---
    # 我们分别缩放 X (特征) 和 y (目标)
    scaler_X = StandardScaler().fit(X_train)
    # y 需要是 (N, 1) 形状才能被缩放
    scaler_y = StandardScaler().fit(y_train.reshape(-1, 1))

    X_train_scaled = scaler_X.transform(X_train)
    y_train_scaled = scaler_y.transform(y_train.reshape(-1, 1)).ravel() # .ravel() 变回 (N,)
    
    # 稍后我们也需要缩放可视化网格
    X_pred_input_scaled = scaler_X.transform(X_pred_input)


    # ==========================================================
    # --- 模型 1: SVR (支持向量回归) ---
    # ==========================================================
    print("\n正在训练 SVR...")
    # C: 惩罚参数。值越大，模型对训练数据拟合越紧。
    # gamma: 'scale' 是一个好的默认值，定义了单个训练样本的影响范围
    svr = SVR(kernel='rbf', C=20, gamma='auto')
    svr.fit(X_train_scaled, y_train_scaled) # 使用标准化数据进行训练

    # 预测 (在标准化的X上)
    Z_pred_scaled = svr.predict(X_pred_input_scaled)
    
    # **关键**: 将预测结果 Z 反向转换回原始尺度
    Z_pred_svr = scaler_y.inverse_transform(Z_pred_scaled.reshape(-1, 1))
    
    # 变回 100x100 网格
    Z_vis_svr = Z_pred_svr.reshape(X_vis.shape)

    # --- 绘制 SVR ---
    fig1 = plt.figure(figsize=(14, 10))
    ax1 = fig1.add_subplot(111, projection='3d')
    ax1.plot_surface(X_vis, Y_vis, Z_vis_svr, cmap='viridis', alpha=0.8, 
                     rstride=1, cstride=1, edgecolor='none')
    ax1.scatter(xs, ys, zs, c='red', s=40, depthshade=True, label='原始数据点')
    ax1.set_title(f'SVR (RBF 核) 拟合曲面', fontsize=18, pad=20)
    ax1.set_xlabel('X 轴 (FLP 表达量)', fontsize=12, labelpad=15)
    ax1.set_ylabel('Y 轴 (Raf 表达量)', fontsize=12, labelpad=15)
    ax1.set_zlabel('Z 轴 (预测数值)', fontsize=12, labelpad=15)
    ax1.legend()
    ax1.view_init(elev=20, azim=-120)


# ==========================================================
    # --- 模型 2: GPR (高斯过程回归) - 调参平滑版 ---
    # ==========================================================
    print("\n正在训练 GPR (调参平滑版)...")
    
    # *** 关键调参 ***
    # 我们将调大 RBF 的 'length_scale' (增加平滑度)
    # 和 WhiteKernel 的 'noise_level' (降低对异常点的敏感度)

    # 1. RBF: length_scale_bounds=(0.5, 1e3) 
    #    我们强制 length_scale (影响范围) 至少为 0.5 (在标准化空间中)
    #    而不是之前的 1e-2 (0.01)，这会防止出现尖峰。
    #    我们将初始猜测 length_scale 设为 5.0 来鼓励平滑。
    rbf_kernel = 1.0 * RBF(length_scale=5.0, length_scale_bounds=(1.0, 1e3))
    
    # 2. WhiteKernel: noise_level_bounds=(0.1, 1e1)
    #    我们强制 noise_level (噪声水平) 至少为 0.1。
    #    这告诉模型 "数据本身是有噪声的，你不需要完美穿过每一个点"。
    #    这会极大地平滑掉那些由异常点引起的峰值。
    white_kernel = WhiteKernel(noise_level=0.5, noise_level_bounds=(0.2, 1e1))
    
    # 将两个核组合起来
    kernel = rbf_kernel + white_kernel
    
    gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
    
    # 训练 (使用标准化的 X 和 y)
    gpr.fit(X_train_scaled, y_train_scaled) 

    # (可选) 打印优化后的内核参数，这很有用
    print(f"GPR 优化后的内核: {gpr.kernel_}")

    # 预测 (在标准化的X上)
    Z_pred_gpr_scaled, Z_std_gpr_scaled = gpr.predict(X_pred_input_scaled, return_std=True)
    
    # 反向转换回原始尺度
    Z_pred_gpr = scaler_y.inverse_transform(Z_pred_gpr_scaled.reshape(-1, 1))
    
    # 变回 100x100 网格
    Z_vis_gpr = Z_pred_gpr.reshape(X_vis.shape)

    # --- 绘制 GPR ---
    fig2 = plt.figure(figsize=(14, 10))
    ax2 = fig2.add_subplot(111, projection='3d')
    ax2.plot_surface(X_vis, Y_vis, Z_vis_gpr, cmap='plasma', alpha=0.8, 
                     rstride=1, cstride=1, edgecolor='none')
    ax2.scatter(xs, ys, zs, c='red', s=40, depthshade=True, label='原始数据点')
    ax2.set_title(f'GPR (高斯过程) 拟合曲面 - 平滑调参版', fontsize=18, pad=20)
    ax2.set_xlabel('X 轴 (FLP 表达量)', fontsize=12, labelpad=15)
    ax2.set_ylabel('Y 轴 (Raf 表达量)', fontsize=12, labelpad=15)
    ax2.set_zlabel('Z 轴 (预测数值)', fontsize=12, labelpad=15)
    ax2.legend()
    ax2.view_init(elev=20, azim=-120)
    plt.show()

except FileNotFoundError:
    print(f"错误：未找到文件 '{file_path}'。")
    print("请确保您的Excel文件与Python脚本在同一目录下。")
except Exception as e:
    print(f"处理数据或绘图时发生错误：{e}")
    print("请检查您的Excel文件格式是否与代码中的 'iloc' 索引匹配。")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from gplearn.genetic import SymbolicRegressor
import graphviz # 用于可视化公式树

# --- 0. 开启 Matplotlib 交互模式 (重要！) ---
# %matplotlib widget 

# --- 1. 配置matplotlib以支持中文 ---
try:
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False 
except Exception as e:
    print(f"警告：设置中文字体失败，标签可能无法正常显示。错误: {e}")

# --- 2. 加载和解析数据 (使用您指定的索引) ---
file_path = '汇总2.xlsx' 
try:
    df = pd.read_excel(file_path, header=None)

    x_labels_raw = df.iloc[1, 2:]
    x_indices = x_labels_raw.notna()
    x_labels = x_labels_raw[x_indices].values.astype(float)
    
    y_labels_raw = df.iloc[2:, 1]
    y_indices = y_labels_raw.notna()
    y_labels = y_labels_raw[y_indices].values.astype(float)
    
    z_data = df.iloc[y_indices[y_indices].index, 
                     x_indices[x_indices].index].values.astype(float)

    print(f"数据加载成功: X轴 {len(x_labels)} 个点, Y轴 {len(y_labels)} 个点, Z数据 {z_data.shape}")

    # --- 3. 准备训练数据 ---
    X_grid, Y_grid = np.meshgrid(x_labels, y_labels)
    xs = X_grid.flatten()
    ys = Y_grid.flatten()
    zs = z_data.flatten()

    X_train = np.stack([xs, ys], axis=1) # (N, 2)
    y_train = zs                         # (N,)
    
    # --- 4. 训练符号回归模型（优化参数以减少内存使用）---
    print("\n--- 正在运行符号回归 (Genetic Programming) ---")
    print("这可能需要几分钟，请稍候...")
    
    # 配置"数学积木" - 减少复杂函数以降低计算负担
    function_set = ['add', 'sub', 'mul', 'div', 'sqrt']  # 移除 sin, cos, log 以加速
    
    est_gp = SymbolicRegressor(
        population_size=500,      # 从 2000 降到 500（减少内存）
        generations=15,           # 从 30 降到 15（加速训练）
        stopping_criteria=10,    
        function_set=function_set,
        metric='mean absolute error',
        p_crossover=0.7,
        p_subtree_mutation=0.1,
        p_hoist_mutation=0.05,
        p_point_mutation=0.1,
        max_samples=0.9,
        verbose=1,
        n_jobs=4,                 # 从 -1 改为 4（限制并行数，避免过载）
        random_state=42
    )

    # -----------------------------------------------------------------
    # --- 终极猴子补丁 (HACK) V3.0 - 完全兼容版 ---
    #
    # 解释：gplearn 0.4.2 与 sklearn 1.7.2 有多个兼容性问题：
    # 1. sklearn 1.7.2 删除了 _validate_data 方法
    # 2. gplearn 的 predict 需要 n_features_in_ 属性
    # 我们必须手动修复这两个问题
    #
    import types
    from sklearn.utils.validation import check_X_y, check_array

    # 修复 1: 定义 _validate_data 函数
    def _our_own_validate_data(self, X, y=None, y_numeric=True, ensure_min_samples=2):
        if y is not None:
            # 训练时：检查 X 和 y
            X_checked, y_checked = check_X_y(X, y, y_numeric=y_numeric, 
                                             ensure_min_samples=ensure_min_samples)
            # 关键：设置 n_features_in_ 属性
            self.n_features_in_ = X_checked.shape[1]
            return X_checked, y_checked
        else:
            # 预测时：只检查 X
            X_checked = check_array(X)
            return X_checked

    # 修复 2: 定义 _validate_params (sklearn 1.7.2 需要)
    def _dummy_validate_params(self):
        pass

    # 3. 把这两个函数"绑"到 est_gp 实例上
    est_gp._validate_data = types.MethodType(_our_own_validate_data, est_gp)
    est_gp._validate_params = types.MethodType(_dummy_validate_params, est_gp)
    #
    # --- 补丁结束 ---
    # -----------------------------------------------------------------

    # 现在再运行 .fit()，所有兼容性问题都解决了
    est_gp.fit(X_train, y_train)
    
    # --- 5. 【请求结果】打印找到的最佳公式 ---
    print("\n--- 符号回归完成 ---")
    print("找到的最佳公式 (Lisp 格式):")
    print(est_gp._program)
    print("---------------------------------")

    # --- 6. 将其填充到空间中 (可视化) - 降低分辨率以节省内存 ---
    
    # a. 创建可视化网格（从 100x100 降到 50x50，减少75%的点）
    x_range = np.linspace(xs.min(), xs.max(), 50)  # 从 100 降到 50
    y_range = np.linspace(ys.min(), ys.max(), 50)  # 从 100 降到 50
    X_vis, Y_vis = np.meshgrid(x_range, y_range)
    X_pred_input = np.stack([X_vis.flatten(), Y_vis.flatten()], axis=1)
    
    # b. 使用找到的公式进行预测
    print("正在生成预测点...")
    Z_pred_gp = est_gp.predict(X_pred_input)
    
    # c. 展平预测结果用于绘制散点图
    x_pred_flat = X_vis.flatten()
    y_pred_flat = Y_vis.flatten()
    z_pred_flat = Z_pred_gp.flatten()

    # --- 7. 绘制 3D 散点图（仅数据点） ---
    print("正在绘制图表...")
    fig = plt.figure(figsize=(12, 9))  # 从 (14, 10) 略微缩小
    ax = fig.add_subplot(111, projection='3d')
    
    # 绘制预测点（模型生成的空间点，用浅色小点）
    ax.scatter(x_pred_flat, y_pred_flat, z_pred_flat, 
               c=z_pred_flat, cmap='coolwarm', s=8, alpha=0.4, 
               depthshade=True, label='模型预测点')
    
    # 绘制原始数据点（用黑色大点突出显示）
    ax.scatter(xs, ys, zs, c='black', s=60, depthshade=True, 
               edgecolors='yellow', linewidths=1.5, label='原始数据点')
    
    ax.set_title(f'符号回归 3D散点图（仅数据点）', fontsize=16, pad=15)
    ax.set_xlabel('X 轴 (FLP 表达量 - X0)', fontsize=11, labelpad=12)
    ax.set_ylabel('Y 轴 (Raf 表达量 - X1)', fontsize=11, labelpad=12)
    ax.set_zlabel('Z 轴 (预测数值)', fontsize=11, labelpad=12)
    ax.legend()
    ax.view_init(elev=20, azim=-120) 
    plt.show()
    print("图表绘制完成！")

    # --- 8. (可选) 可视化公式树 ---
    print("\n正在生成公式树... (将保存为 'symbolic_formula.png')")
    try:
        dot_data = est_gp._program.export_graphviz()
        graph = graphviz.Source(dot_data)
        # 保存为文件。在 Notebook 中也可以直接输入 graph 变量来显示
        graph.render('symbolic_formula', format='png', cleanup=True)
        print("公式树已保存！")
    except Exception as e:
        print(f"无法生成公式树。请确保您已安装 Graphviz 可执行文件。错误: {e}")

except FileNotFoundError:
    print(f"错误：未找到文件 '{file_path}'。")
except Exception as e:
    print(f"处理数据或绘图时发生错误：{e}")
    import traceback
    traceback.print_exc()  # 打印详细错误信息以便调试

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 配置中文显示
try:
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
except:
    pass

file_path = '汇总2.xlsx'

try:
    # 读取数据
    df = pd.read_excel(file_path, header=None)
    
    x_labels_raw = df.iloc[1, 2:]
    x_indices = x_labels_raw.notna()
    x_labels = x_labels_raw[x_indices].values.astype(float)
    
    y_labels_raw = df.iloc[2:, 1]
    y_indices = y_labels_raw.notna()
    y_labels = y_labels_raw[y_indices].values.astype(float)
    
    z_data = df.iloc[y_indices[y_indices].index, 
                     x_indices[x_indices].index].values.astype(float)
    
    print(f"数据加载: X轴 {len(x_labels)} 点, Y轴 {len(y_labels)} 点, Z数据 {z_data.shape}")
    
    # 创建网格
    X_grid, Y_grid = np.meshgrid(x_labels, y_labels)
    xs = X_grid.flatten()
    ys = Y_grid.flatten()
    zs = z_data.flatten()
    
    # ========== 创建3D投影图 ==========
    fig = plt.figure(figsize=(16, 12))
    
    # 主3D图
    ax_3d = fig.add_subplot(2, 2, 1, projection='3d')
    scatter = ax_3d.scatter(xs, ys, zs, c=zs, cmap='viridis', 
                            s=80, alpha=0.8, edgecolors='black', linewidths=1)
    ax_3d.set_xlabel('FLP 表达量 (X)', fontsize=11, labelpad=10)
    ax_3d.set_ylabel('Raf 表达量 (Y)', fontsize=11, labelpad=10)
    ax_3d.set_zlabel('数值 (Z)', fontsize=11, labelpad=10)
    ax_3d.set_title('3D 数据点分布', fontsize=14, pad=15)
    ax_3d.view_init(elev=25, azim=-60)
    plt.colorbar(scatter, ax=ax_3d, shrink=0.5, label='Z值')
    
    # X-Y投影 (俯视图)
    ax_xy = fig.add_subplot(2, 2, 2)
    scatter_xy = ax_xy.scatter(xs, ys, c=zs, cmap='viridis', 
                               s=100, alpha=0.7, edgecolors='black', linewidths=1)
    ax_xy.set_xlabel('FLP 表达量 (X)', fontsize=11)
    ax_xy.set_ylabel('Raf 表达量 (Y)', fontsize=11)
    ax_xy.set_title('XY 平面投影 (俯视)', fontsize=12)
    ax_xy.grid(True, alpha=0.3)
    plt.colorbar(scatter_xy, ax=ax_xy, label='Z值')
    
    # X-Z投影 (侧视图)
    ax_xz = fig.add_subplot(2, 2, 3)
    scatter_xz = ax_xz.scatter(xs, zs, c=zs, cmap='plasma', 
                               s=100, alpha=0.7, edgecolors='black', linewidths=1)
    ax_xz.set_xlabel('FLP 表达量 (X)', fontsize=11)
    ax_xz.set_ylabel('数值 (Z)', fontsize=11)
    ax_xz.set_title('XZ 平面投影 (侧视)', fontsize=12)
    ax_xz.grid(True, alpha=0.3)
    plt.colorbar(scatter_xz, ax=ax_xz, label='Z值')
    
    # Y-Z投影 (正视图)
    ax_yz = fig.add_subplot(2, 2, 4)
    scatter_yz = ax_yz.scatter(ys, zs, c=zs, cmap='coolwarm', 
                               s=100, alpha=0.7, edgecolors='black', linewidths=1)
    ax_yz.set_xlabel('Raf 表达量 (Y)', fontsize=11)
    ax_yz.set_ylabel('数值 (Z)', fontsize=11)
    ax_yz.set_title('YZ 平面投影 (正视)', fontsize=12)
    ax_yz.grid(True, alpha=0.3)
    plt.colorbar(scatter_yz, ax=ax_yz, label='Z值')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ 3D投影图绘制完成!")
    print(f"  - 左上: 3D立体图")
    print(f"  - 右上: XY平面投影 (俯视)")
    print(f"  - 左下: XZ平面投影 (侧视)")
    print(f"  - 右下: YZ平面投影 (正视)")

except FileNotFoundError:
    print(f"错误: 未找到文件 '{file_path}'")
except Exception as e:
    print(f"错误: {e}")
    import traceback
    traceback.print_exc()

In [1]:
def func(a=None, b=[]):
    if a is None:
        print(f"a is still None (Immutable)")
    else:
        print(f"a is now {a} (New local variable)")
    
    b.append(100)
    print(f"b list ID: {id(b)}")
    print(f"b: {b}")

print(f"Function defaults b list ID: {id(func.__defaults__[1])}")
print("-" * 20)

# 第一次调用
func()
# b: [100]

# 第二次调用（a 还是 None，b 却带着上次修改的值）
func()
# b: [100, 100] 

# 传入新的列表，b 的 ID 改变
func(b=[])
# b: [100] (这是新的列表)

# 第三次调用（不传 b，b 再次使用那个共享的、被修改过的列表）
func()
# b: [100, 100, 100]

Function defaults b list ID: 2830221234560
--------------------
a is still None (Immutable)
b list ID: 2830221234560
b: [100]
a is still None (Immutable)
b list ID: 2830221234560
b: [100, 100]
a is still None (Immutable)
b list ID: 2830221232512
b: [100]
a is still None (Immutable)
b list ID: 2830221234560
b: [100, 100, 100]


In [7]:
dir(func)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [21]:
dir(func)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [ ]:
a

In [11]:
class A():
    k=[]
    def l(self):
        print("a")
    def __init__(self):
        self.l()
        self.k.append(1)

a=A()

a


In [2]:
a=[4,5]

b=list(a)
b.append(6)
a

[4, 5]

In [8]:
b=[4,5,5,6,5]

In [9]:
b.remove(5)
b

[4, 5, 6, 5]

In [12]:
b.insert(2,5)

In [30]:
f=open("try.txt",'r',encoding="utf-8")
k=f.read()
print("你好\n好的")

你好
好的


In [1]:
import torch

# 1. 原始张量 A，需要梯度
A = torch.tensor([2.0, 3.0], requires_grad=True)

# 2. 原始运算，形成 A 的历史
B = A * 2 
# B 继承了 A 的历史，B.grad_fn 存在

# ----------------------------------------------------
# 目标：创建一个 C，保留 B 的历史，但 C 之后的计算是新的分支
# ----------------------------------------------------

# **方法一：使用 .clone()**
# 默认情况下，.clone() 会继承 requires_grad=True
C = B.clone()

# C 现在是计算图上的一个新节点，但它依赖于 B (通过 clone 操作)
# C.grad_fn 存在，指向 CloneBackward
# C 的数据是 B 的副本

# 3. 在 C 上进行后续运算（分叉）
D = C + 5
# D 依赖于 C。当计算 D 的梯度时，它会通过 C 回传到 B，再回传到 A。

# ----------------------------------------------------
# 验证：执行反向传播
D_sum = D.sum()
D_sum.backward()

# 验证结果：
print(f"A 的梯度: {A.grad}") # A 应该有梯度，因为历史被保留了
# A 的梯度: tensor([1., 1.])

A 的梯度: tensor([2., 2.])


In [6]:
B = torch.tensor([2.0, 3.0])
A=B*2
B.requires_grad

False

In [7]:
0==0.0

True

In [19]:
print("{1}ok{0},好啊{2}".format(*{1:4,5:8,"什么":88}.items()))

(5, 8)ok(1, 4),好啊('什么', 88)


In [21]:
async def F():
    print("hello")

await F()

hello


In [30]:
for i in iter(range(10)):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [49]:
import json
dict_json={}
with open("try.json",'r',encoding='utf-8')  as f:
    dict_json=json.load(f)
    dict_json['level']+=1
    dict_json['gold']+=50
    dict_json['inventory'].append("shield")
    json.dump(dict_json,f)

UnsupportedOperation: not writable

In [41]:
import csv
sum_all=0
with open("try.csv",'r',encoding="utf-8") as f:
    a=csv.DictReader(f)
    for row in a:
        sum_all+=int(row['Price'])*int(row['Count'])

sum_all

37

In [50]:
with open("try.txt",'r+',encoding='utf-8') as f:
    a=f.read()

a

'apple, 10\nbanana, 5\norange, 8\ngrape, 20\napple, 5'

In [27]:
with open("try.txt",'r',encoding='utf-8') as f:
    a=f.readline()
    fruit={}
    while(a!=''):
        a=a.strip().lower().split(",")[0]
        if a not in fruit.keys():
            fruit[a]=1
        else:
            fruit[a]+=1
        print(fruit)
        a=f.readline()


{'apple': 1}
{'apple': 1, 'banana': 1}
{'apple': 1, 'banana': 1, 'orange': 1}
{'apple': 1, 'banana': 1, 'orange': 1, 'grape': 1}
{'apple': 2, 'banana': 1, 'orange': 1, 'grape': 1}


In [28]:
with open("try.txt",'r',encoding='utf-8') as f:
    a=f.readline()
    while(a!=''):
        a=a.strip().lower()
        print(a)
        a=f.readline()


apple, 10
banana, 5
orange, 8
grape, 20
apple, 5


In [75]:
import os

# 生成一些假文件
files = ["exp1.txt", "exp2.txt", "config.json", "data.csv", "log.txt", "model.json"]
for f_name in files:
    with open(f_name, 'w') as f:
        f.write("dummy content")
print("测试文件已生成。")

测试文件已生成。


In [80]:
if not os.path.exists("text_data"):
    os.mkdir("text_data")
if not os.path.exists("json_data"):
    os.mkdir("json_data")

list_dir=os.listdir()
for i,j in zip(list_dir,map(lambda x:os.path.splitext(x)[1],list_dir)):
    if j=='.txt':
        os.rename(i,os.path.join("text_data",i))
    if j=='.json':
        os.rename(i,os.path.join("json_data",i))

In [83]:
import shutil
shutil.copytree("text_data","what")

'what'

In [92]:
os.path.dirname('build\\CMakeFiles\\TargetDirectories.txt')

'build\\CMakeFiles'

In [56]:
import os
os.path.split(os.path.splitext("user\data\wat.jpg")[0])

('user\\data', 'wat')

In [7]:
import numpy as np
a=np.array([[4,5,5,6],[5,6,5,8],[4,8,9,5]])
a[a>5].shape

(5,)

In [94]:
"hello,   worlg\n".strip()

'hello,   worlg'

In [100]:
class Link:
    """A linked list.

    >>> s = Link(1)
    >>> s.first
    1
    >>> s.rest is Link.empty
    True
    >>> s = Link(2, Link(3, Link(4)))
    >>> s.first = 5
    >>> s.rest.first = 6
    >>> s.rest.rest = Link.empty
    >>> s                                    # Displays the contents of repr(s)
    Link(5, Link(6))
    >>> s.rest = Link(7, Link(Link(8, Link(9))))
    >>> s
    Link(5, Link(7, Link(Link(8, Link(9)))))
    >>> print(s)                             # Prints str(s)
    <5 7 <8 9>>
    """
    empty = ()
    def __init__(self, first, rest=empty):
        assert rest is Link.empty or isinstance(rest, Link)
        self.first = first
        self.rest = rest

    def __repr__(self):
        if self.rest is not Link.empty:
            rest_repr = ', ' + repr(self.rest)
        else:
            rest_repr = ''
        return 'Link(' + repr(self.first) + rest_repr + ')'

    def __str__(self):
        string = '<'
        while self.rest is not Link.empty:
            string += str(self.first) + ' '
            self = self.rest
        return string + str(self.first) + '>'




In [110]:
a=[45,4,65,5]
a.copy()

[45, 4, 65, 5]

In [112]:
import copy
copy.deepcopy(a)

[45, 4, 65, 5]

In [119]:
b=Link(2)
s=Link(1,b)
a=copy.deepcopy(s)


In [7]:
class Tree:
    """A tree has a label and a list of branches.

    >>> t = Tree(3, [Tree(2, [Tree(5)]), Tree(4)])
    >>> t.label
    3
    >>> t.branches[0].label
    2
    >>> t.branches[1].is_leaf()
    True
    """
    def __init__(self, label, branches=[]):
        self.label = label
        for branch in branches:
            assert isinstance(branch, Tree)
        self.branches = list(branches)

    def is_leaf(self):
        return not self.branches

    def __repr__(self):
        if self.branches:
            branch_str = ', ' + repr(self.branches)
        else:
            branch_str = ''
        return 'Tree({0}{1})'.format(repr(self.label), branch_str)

    def __str__(self):
        return '\n'.join(self.indented())

    def indented(self):
        lines = []
        for b in self.branches:
            for line in b.indented():
                lines.append('  ' + line)
        return [str(self.label)] + lines
    
t=Tree(0)
t

Tree(0)

In [21]:
index=1
def f():
    a=index
f()
index
f.__closure__

In [40]:
index=1
def f():
    def g():
        global index
        index+=1
        return index
    return g
f()()

2

In [41]:
def make_counter():
    # 使用一个列表作为容器，存储计数值
    count_container = [0] 
    
    def increment():
        # 我们没有重新赋值 count_container，只是修改了它指向的列表的内容
        count_container[0] += 1
        return count_container[0]
        
    return increment

counter = make_counter()

print(counter()) # 输出: 1
print(counter()) # 输出: 2

1
2


In [62]:
a=(4 ,5 ,6)
a[0]

4

In [52]:
a=[4 ,5 ,[5, 6 ]]
b=a[2]
c=a[:]
b.append(6)
c.append(6)
a

[4, 5, [5, 6, 6]]

In [ ]:
class nil:
    alll=2
    def g():
        print("hello")
        
    def __init__(self):
        self.alll=3

a=nil()

hello


In [121]:
b.first=4

In [80]:
import scipy
j=10
value_and_prob_j={}
for k in range(2*j,6*j+1):
    N=0
    for n in range((k-2*j)//5+1):
        N+=(-1)**n*scipy.special.comb(j,n)*scipy.special.comb(k-j-5*n-1,j-1)
    value_and_prob_j[k]=N/(5**j)


In [87]:
string_input='A paragraph about dogs.'
new=''.join(c for c in string_input if c.isalpha())
new=new.lower().split(' ')
new

['aparagraphaboutdogs']

In [94]:
a={}
a[1+1]=2
a

{2: 2}

In [102]:
len("")

0

In [101]:
"'".lower()

"'"

In [100]:
typed="s'"
temp=typed[1]+typed[0]+typed[2:]
temp

"'s"

In [3]:
a=list(range(10))
for i in a.copy():
    a.remove(i)

In [17]:
class FileProcessingError(Exception):
    def __init__(self, message, filename, lineno):
        super().__init__(message) # 调用父类构造函数设置主要的错误信息
        self.filename = filename
        self.lineno = lineno

# 抛出时：
try:
    FileProcessingError("文件格式错误", "data.txt", 15)
except FileProcessingError("文件格式错误", "data.txt", 15) as e:
    print(e.filename)

In [65]:
def piii(length=2):
    
    piii.num-=1
    pass
piii.num=0

In [ ]:
{a:1}

TypeError: unhashable type: 'dict'

In [88]:
print('\"')

"


In [85]:
def f(*args,k=1):
    num=0
    for i in args:
        num+=i
    return num
f(*([4,5,6]+[4,5,6]),k=3)

30

In [24]:
import random
random.choice(["ac","asc","avsve"])

'avsve'

In [33]:
g=lambda self,game :  None
g(1,2)

Correction Speed: 915.4297033615109 wpm
Correctly Corrected: 440 words
Incorrectly Corrected: 196 words
Uncorrected: 51 words

Correction Speed: 1245.1672966717474 wpm
Correctly Corrected: 578 words
Incorrectly Corrected: 286 words
Uncorrected: 70 words

Correction Speed: 505.5774036685697 wpm
Correctly Corrected: 265 words
Incorrectly Corrected: 97 words
Uncorrected: 21 words

Correction Speed: 691.6608272363577 wpm
Correctly Corrected: 333 words
Incorrectly Corrected: 148 words
Uncorrected: 41 words

Correction Speed: 92704.22414664713 wpm
Correctly Corrected: 428 words
Incorrectly Corrected: 400 words
Uncorrected: 112 words

'hello world'

'range(0, 10)'

In [24]:
import json
for file in open("try.json"):
    f=json.loads(file)

f

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [93]:
def fact(n,result):
    def tunk():
        if n==0:
            return result
        return fact(n-1,result*n)
    return tunk


def fact_finla(n):
    val=fact(n,1)
    while(callable(val)):
        val=val()
    return val

fact_finla(10)


3628800

In [91]:
callable(1)

False